# **Project : A Case Study of ExpressWay Logistics**

**Business Overview:**

ExpressWay Logistics is a dynamic logistics service provider, committed to delivering efficient, reliable and cost-effective courier transportation and warehousing solutions. With a focus on speed, precision and customer satisfaction, we aim to be the go-to partner for our customers seeking seamless courier services. Our core service involves ensuring operational efficiency throughout our delivery and courier services, including inventory management, durable packaging and swift dispatch of couriers, real time tracking of shipments and on-time delivery of couriers as promised. We are committed to enhance our logistics and courier services and improve seamless connectivity for our customers.

**Current Challenge:**

ExpressWay Logistics faces numerous challenges in ensuring seamless deliveries and customer satisfaction. These challenges include managing various customer demands simultaneously, addressing delays in deliveries and ensuring products arrive intact and safe. Additionally, the company struggles with complexity of efficiently storing and handling a large volume of packages and ultimately meeting customer expectations. Moreover, maintaining a skilled workforce capable of handling various aspects of logistics operations presents its own set of challenges. Overcoming these obstacles requires a comprehensive approach that integrates innovative technology, strategic planning, and continuous improvement initiatives to ensure smooth operations and exceptional service delivery.

**Objective:**

Our primary objective is to conduct a sentiment analysis of user-generated reviews across various digital channels and platforms. By paying attention to their feedback, we want to find ways to make our services better - like handling different customer demands simultaneously, dealing with late deliveries, and keeping packages secured and intact. Through the application of prompt engineering methodologies and sentiment analysis, we'll figure out if sentiments expressed by users for our courier services are Positive or Negative. This will help us understand where we need to improve in order to meet customer expectations and keep them happy. With a focus on getting better all the time, we'll overcome the challenges at ExpressWay Logistics and make our services the best.

**Data Description:**

The dataset titled "courier-service_reviews.csv" is structured to facilitate sentiment analysis for courier service reviews. Here's a brief description of the data columns:

1. id: This column contains unique identifiers for each review entry. It helps in distinguishing and referencing individual reviews.
2. review: This column includes the actual text of the courier service reviews. The reviews are likely composed of customer opinions and experiences regarding different aspects of the services provided by ExpressWay Logistics.
3. sentiment: This column provides an additional layer of classification (positive and negative) for the mentioned reviews.

##**Step 1. Setup (2 Marks)**

(A) Writing/Creating the config.json file  (2 Marks)

### Installation

In [38]:
!pip install openai==1.2 tiktoken datasets session-info --quiet

### Imports

In [3]:
# Import all Python packages required to access the Azure Open AI API.
# Import additional packages required to access datasets and create examples.

from openai import AzureOpenAI
import json
import random
import tiktoken
import session_info

import pandas as pd
import numpy as np

from collections import Counter
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tabulate import tabulate

In [40]:
session_info.show()

### Authentication

In [1]:
with open('config.json', 'r') as az_creds:
    data = az_creds.read()

In [4]:
creds = json.loads(data)

In [ ]:
# Credentials to authenticate to the personalized Open AI model server

client = AzureOpenAI(
    azure_endpoint=creds["AZURE_OPENAI_ENDPOINT"],
    api_key=creds["AZURE_OPENAI_KEY"],
    api_version=creds["AZURE_OPENAI_APIVERSION"]
)

In [46]:
chat_model_id = creds["CHATGPT_MODEL"]

In [47]:
chat_model_id

'sridharmls04'

In [48]:
deployment_name = creds["CHATGPT_MODEL"]
deployment_name

'sridharmls04'

### Utilities

In [49]:
def num_tokens_from_messages(messages):

    """
    Return the number of tokens used by a list of messages.
    Adapted from the Open AI cookbook token counter
    """

    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

    # Each message is sandwiched with <|start|>role and <|end|>
    # Hence, messages look like: <|start|>system or user or assistant{message}<|end|>

    tokens_per_message = 3 # token1:<|start|>, token2:system(or user or assistant), token3:<|end|>

    num_tokens = 0

    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))

    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>

    return num_tokens

## Task : Sentiment Analysis

##**Step 2: Assemble Data (5 Marks)**

(A) Upload and Read csv File (2 Marks)

(B) Count Positive and Negative Sentiment Reviews (1 Marks)

(C) Split the Dataset (2 Marks)

**(A) Upload and read csv file (2 Marks)**

In [50]:
 cs_reviews_df = pd.read_csv("courier-service_reviews.csv")
# Read CSV File Here

In [51]:
cs_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         131 non-null    int64 
 1   review     131 non-null    object
 2   sentiment  131 non-null    object
dtypes: int64(1), object(2)
memory usage: 3.2+ KB


In [52]:
cs_reviews_df.sample(5)

,id,review,sentiment
117,118,"When it comes to handling parcels, ExpressWay ...",Positive
53,54,ExpressWay Logistics' team of dedicated custom...,Positive
104,105,ExpressWay Logistics is my go-to for all my sh...,Positive
97,98,"As a small business owner, I appreciate Expres...",Positive
74,75,"I had high hopes for ExpressWay Logistics, but...",Positive


**(B) Count Positive and Negative Sentiment Reviews (1 Marks)**

In [53]:
cs_reviews_df["sentiment"].value_counts()

sentiment
Positive    68
Negative    63
Name: count, dtype: int64

In [54]:
cs_reviews_df.shape

(131, 3)

**(C) Split the Dataset (2 Marks)**

In [55]:
cs_examples_df, cs_gold_examples_df = train_test_split(
    cs_reviews_df, #<- the full dataset
    test_size=0.2, #<- 20% random sample selected for gold examples
    random_state=42 #<- ensures that the splits are the same for every session
)

In [56]:
(cs_examples_df.shape, cs_gold_examples_df.shape)

((104, 3), (27, 3))

To select gold examples for this session, we sample randomly from the test data using a `random_state=42`. This ensures that the examples from multiple runs of the sampling are the same (i.e., they are randomly selected but do not change between different runs of the notebook). Note that we are doing this only to keep execution times low for illustration. In practise, large number of gold examples facilitate robust estimates of model accuracy.

In [57]:
columns_to_select = ['review','sentiment']

In [58]:
gold_examples = (
        cs_gold_examples_df.loc[:, columns_to_select]
                                     .sample(21, random_state=42) #<- ensures that gold examples are the same for every session
                                     .to_json(orient='records')
)

In [59]:
# gold_examples = (
#         cs_gold_examples_df.loc[:, columns_to_select]
#                                      .sample(21) #<- ensures that gold examples are the same for every session
#                                      .to_json(orient='records')
# )

In [60]:
gold_examples

'[{"review":"The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive\'s efforts. The packaging seemed more than adequate to protect the contents during transit.","sentiment":"Positive"},{"review":"ExpressWay Logistics failed to meet my expectations. The delivery was delayed, and the customer support team was unresponsive and unhelpful when I tried to inquire about the status of my parcel.","sentiment":"Negative"},{"review":"ExpressWay Logistics\' incompetence resulted in a major inconvenience when my package was delivered to the wrong recipient. Despite providing accurate delivery information, the package ended up in the hands of someone else, and efforts to retrieve it were unsuccessful. When I contacted customer service for assistance, I was met with apathy and a lack of urgency. Their fa

In [61]:
json.loads(gold_examples)[0]     #Json format

{'review': "The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive's efforts. The packaging seemed more than adequate to protect the contents during transit.",
 'sentiment': 'Positive'}

##**Step 3: Derive Prompt (12 Marks)**

(A) Write Zero Shot System Message (3 Marks)

(B) Create Zero Shot Prompt (2 Marks)

(C) Write Few Shot System Message (3 Marks)

(D) Create Examples For Few shot prompte (2 Marks)

(E) Create Few Shot Prompt (2 Marks)

In [62]:
user_message_template = """```{courier_service_review}```"""

**(A) Write Zero Shot System Message (3 Marks)**

In [63]:
zero_shot_system_message = """Classify the sentiment of the following review presented in the input into one of the following categories.
Categories - ['Positive', 'Negative']
The review will be delimited by triple backticks in the input.
Carefully evaluate sentiment of the review and respond only 'Positive' or 'Negative'.  Do not explain your answer."""
# Write zero shot system message here

**(B) Create Zero Shot Prompt (2 Marks)**

In [64]:
zero_shot_prompt = [{'role':'system', 'content': zero_shot_system_message}]
# Create zero shot prompt to be input ready for completion function

In [65]:
num_tokens_from_messages(zero_shot_prompt)

71

In [66]:
cs_reviews_df.iloc[0,1]

"ExpressWay Logistics' commitment to transparency gives us confidence in their services. They provide clear and upfront pricing, so we know exactly what to expect. With ExpressWay Logistics, there are no hidden fees or surprises, just reliable service at a fair price."

In [67]:
input_description = cs_reviews_df.iloc[0,1]

user_input = [
    {
        'role':'user',
        'content': user_message_template.format(courier_service_review = input_description)
    }
]

In [68]:
response = client.chat.completions.create(
    model=deployment_name,
    messages=zero_shot_prompt+user_input,
    temperature=0.3, # <- the temperature it set to low to make derministic 
    max_tokens=10 # <- restrict the output to not more than 10 tokens
)
print(response.choices[0].message.content)

Positive


**(C) Write Few Shot System Message (3 Marks)**

In [69]:
few_shot_system_message = """Classify the following review presented in the input into one of the following categories.
Categories - ['Positive', 'Negative']
review will be delimited by triple backticks in the input.
Answer only 'Positive' or 'Negative'. Nothing Else. Do not explain your answer."""

Merely selecting random samples from the polarity subsets is not enough because the examples included in a prompt are prone to a set of known biases such as:
 - Majority label bias (frequent answers in predictions)
 - Recency bias (examples near the end of the prompt)


To avoid these biases, it is important to have a balanced set of examples that are arranged in random order. Let us create a Python function that generates bias-free examples:

In [70]:
def create_examples(dataset, n=4):

    """
    Return a JSON list of randomized examples of size 2n with two classes.
    Create subsets of each class, choose random samples from the subsets,
    merge and randomize the order of samples in the merged list.
    Each run of this function creates a different random sample of examples
    chosen from the training data.

    Args:
        dataset (DataFrame): A DataFrame with examples (review + label)
        n (int): number of examples of each class to be selected

    Output:
        randomized_examples (JSON): A JSON with examples in random order
    """

    positive_reviews = (dataset.sentiment == 'Positive')
    negative_reviews = (dataset.sentiment == 'Negative')
    columns_to_select = ['review', 'sentiment']

    positive_examples = dataset.loc[positive_reviews, columns_to_select].sample(n)
    negative_examples = dataset.loc[negative_reviews, columns_to_select].sample(n)

    examples = pd.concat([positive_examples, negative_examples])

    # sampling without replacement is equivalent to random shuffling

    randomized_examples = examples.sample(2*n, replace=False)

    return randomized_examples.to_json(orient='records')

**(D) Create Examples For Few shot prompts (2 Marks)**

In [71]:
examples = create_examples(cs_examples_df, 4)
# Create Examples

In [72]:
json.loads(examples)

[{'review': 'ExpressWay Logistics offers a wide range of shipping options, which is convenient. Despite their convenient online booking platform, frequent delays in delivery and poor communication regarding package status have left me frustrated.Their pricing structure is unclear, with hidden fees often surfacing unexpectedly.Overall they need to improve a lot in their services.',
  'sentiment': 'Negative'},
 {'review': "ExpressWay Logistics' commitment to transparency gives us confidence in their services. They provide clear and upfront pricing, so we know exactly what to expect. With ExpressWay Logistics, there are no hidden fees or surprises, just reliable service at a fair price.",
  'sentiment': 'Positive'},
 {'review': 'ExpressWay Logistics has potential, but they need to focus on improving their operational consistency and pricing transparency to truly stand out in the industry.They are good at customer support ,resolving doubts and queries.I love their efforts in even small thi

With the examples in place, we can now assemble a few-shot prompt. Since we will be using the few-shot prompt several times during evaluation, let us write a function to create a few-shot prompt (the logic of this function is depicted below).

In [73]:
def create_prompt(system_message, examples, user_message_template):

    """
    Return a prompt message in the format expected by the Open AI API.
    Loop through the examples and parse them as user message and assistant
    message.

    Args:
        system_message (str): system message with instructions for sentiment analysis
        examples (str): JSON string with list of examples
        user_message_template (str): string with a placeholder for courier service reviews

    Output:
        few_shot_prompt (List): A list of dictionaries in the Open AI prompt format
    """

    few_shot_prompt = [{'role':'system', 'content': system_message}]

    for example in json.loads(examples):
        example_review = example['review']
        example_sentiment = example['sentiment']

        few_shot_prompt.append(
            {
                'role': 'user',
                'content': user_message_template.format(
                    courier_service_review=example_review
                )
            }
        )

        few_shot_prompt.append(
            {'role': 'assistant', 'content': f"{example_sentiment}"}
        )

    return few_shot_prompt

**(E) Create Few Shot Prompt (2 Marks)**

In [74]:
few_shot_prompt = create_prompt(few_shot_system_message, examples, user_message_template)
# Create Few shot prompt

In [75]:
few_shot_prompt

[{'role': 'system',
  'content': "Classify the following review presented in the input into one of the following categories.\nCategories - ['Positive', 'Negative']\nreview will be delimited by triple backticks in the input.\nAnswer only 'Positive' or 'Negative'. Nothing Else. Do not explain your answer."},
 {'role': 'user',
  'content': '```ExpressWay Logistics offers a wide range of shipping options, which is convenient. Despite their convenient online booking platform, frequent delays in delivery and poor communication regarding package status have left me frustrated.Their pricing structure is unclear, with hidden fees often surfacing unexpectedly.Overall they need to improve a lot in their services.```'},
 {'role': 'assistant', 'content': 'Negative'},
 {'role': 'user',
  'content': "```ExpressWay Logistics' commitment to transparency gives us confidence in their services. They provide clear and upfront pricing, so we know exactly what to expect. With ExpressWay Logistics, there are 

In [76]:
num_tokens_from_messages(few_shot_prompt)

558

##**Step 4: Evaluate prompts (8 Marks)**

(A) Evaluate Zero Shot Prompt (2 Marks)

(B) Evaluate Few Shot Prompt (2 marks)

(C) Calculate Mean and Standard Deviation for Zero Shot Prompt and Few Shot Prompt (4 Marks)

Now we have two sets of prompts that we need to evaluate using gold labels. Since the few-shot prompt depends on the sample of examples that was drawn to make up the prompt, we expect some variability in evaluation. Hence, we evaluate each prompt multiple times to get a sense of the average and the variation around the average.

To reiterate, a choice on the prompt should account for variability due to the choice of the random sample. To aid repeated evaluation, we assemble an evaluation function .

In [77]:
import json
from sklearn.metrics import f1_score
from tabulate import tabulate
from datetime import datetime

def evaluate_prompt(prompt, gold_examples, user_message_template, output_file='output_results.txt'):
    """
    Return the micro-F1 score for predictions on gold examples.
    For each example, we make a prediction using the prompt. Gold labels and
    model predictions are aggregated into lists and compared to compute the
    F1 score.

    Args:
        prompt (List): list of messages in the Open AI prompt format
        gold_examples (str): JSON string with list of gold examples
        user_message_template (str): string with a placeholder for courier service review
        output_file (str): file to write the results

    Output:
        micro_f1_score (float): Micro-F1 score computed by comparing model predictions
                                with ground truth
    """

    start_time = datetime.now()
    with open(output_file, 'a') as f:
        f.write(f"Evaluation Start Time: {start_time.strftime('%Y-%m-%d %H:%M:%S')}\n")

    model_predictions, ground_truths, review_texts = [], [], []

    for example in json.loads(gold_examples):
        gold_input = example['review']
        user_input = [
            {
                'role': 'user',
                'content': user_message_template.format(courier_service_review=gold_input)
            }
        ]

        try:
            response = client.chat.completions.create(
                model=chat_model_id,
                messages=prompt + user_input,
                temperature=0.3,
                max_tokens=10
            )

            prediction = response.choices[0].message.content
            model_predictions.append(prediction.strip())
            ground_truths.append(example['sentiment'])
            review_texts.append(gold_input)
            #print(f"API response: {response}")  # Log the entire response
        except Exception as e:
            print(f"Error: {e}", flush=True)
            continue

    micro_f1_score = f1_score(ground_truths, model_predictions, average="micro")

    table_data = [[text, pred, truth] for text, pred, truth in zip(review_texts, model_predictions, ground_truths)]
    headers = ["Review", "Model Prediction", "Ground Truth"]
    table_output = tabulate(table_data, headers=headers, tablefmt="grid")

    # Writing to file
    with open(output_file, 'a') as f:
        f.write(table_output + "\n")
        f.write(f"Micro F1 Score: {micro_f1_score:.4f}\n")

    # Calculate accuracy and percentages
    correct_predictions = sum(pred == truth for pred, truth in zip(model_predictions, ground_truths))
    total_predictions = len(model_predictions)

    accuracy_percentage = (correct_predictions / total_predictions) * 100

    # Output the results
    results_output = (
        f"Correct Predictions: {correct_predictions}/{total_predictions}\n"
        f"Accuracy Percentage: {accuracy_percentage:.2f}%\n"
    )

    with open(output_file, 'a') as f:
        f.write(results_output)

    total_reviews = len(model_predictions)
    positive_predictions = sum(pred == "Positive" for pred in model_predictions)
    negative_predictions = total_reviews - positive_predictions

    positive_ground_truths = sum(truth == "Positive" for truth in ground_truths)
    negative_ground_truths = total_reviews - positive_ground_truths

    # Writing additional stats to file
    stats_output = (
        f"Total Reviews: {total_reviews}\n"
        f"Positive Predictions: {positive_predictions} ({(positive_predictions / total_reviews) * 100:.2f}%)\n"
        f"Negative Predictions: {negative_predictions} ({(negative_predictions / total_reviews) * 100:.2f}%)\n"
        f"Positive Ground Truths: {positive_ground_truths} ({(positive_ground_truths / total_reviews) * 100:.2f}%)\n"
        f"Negative Ground Truths: {negative_ground_truths} ({(negative_ground_truths / total_reviews) * 100:.2f}%)\n"
    )

    with open(output_file, 'a') as f:
        f.write(stats_output)

    end_time = datetime.now()
    with open(output_file, 'a') as f:
        f.write(f"Evaluation End Time: {end_time.strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Duration: {(end_time - start_time).total_seconds()} seconds\n\n")

    return micro_f1_score

Let us now use this function to do one evaluation of all the two prompts assembled so far, each time computing the Micro-F1 score.

**(A) Evaluate zero shot prompt (2 Marks)**

In [78]:
evaluate_prompt(zero_shot_prompt, gold_examples, user_message_template)

0.9047619047619048

**(B) Evaluate few shot prompt (2 Marks)**

In [79]:
evaluate_prompt(few_shot_prompt, gold_examples, user_message_template)

0.9523809523809523

However, this is just *one* choice of examples. We will need to run these evaluations with multiple choices of examples to get a sense of variability in F1 score for the few-shot prompt. As an example, let us run evaluations for the few-shot prompt 5 times.

In [80]:
num_eval_runs = 5

In [81]:
zero_shot_performance = []
few_shot_performance = []

In [82]:
for _ in tqdm(range(num_eval_runs)):

    # For each run create a new sample of examples
    examples = create_examples(cs_examples_df)

    # Assemble the zero shot prompt with these examples
    zero_shot_prompt = [{'role':'system', 'content': zero_shot_system_message}]
    # zero_shot_prompt = create_prompt(zero_shot_system_message, examples, user_message_template)
    print('zero_shot_prompt:',zero_shot_prompt)
    
    # Assemble the few shot prompt with these examples
    few_shot_prompt = create_prompt(few_shot_system_message, examples, user_message_template)
    print('few_shot_prompt:',few_shot_prompt)
    
    # Evaluate zero shot prompt accuracy on gold examples
    zero_shot_micro_f1 = evaluate_prompt(zero_shot_prompt, gold_examples, user_message_template)

    # Evaluate few shot prompt accuracy on gold examples
    few_shot_micro_f1 = evaluate_prompt(few_shot_prompt, gold_examples, user_message_template)

    zero_shot_performance.append(zero_shot_micro_f1)
    few_shot_performance.append(few_shot_micro_f1)

  0%|          | 0/5 [00:00<?, ?it/s]

zero_shot_prompt: [{'role': 'system', 'content': "Classify the sentiment of the following review presented in the input into one of the following categories.\nCategories - ['Positive', 'Negative']\nThe review will be delimited by triple backticks in the input.\nCarefully evaluate sentiment of the review and respond only 'Positive' or 'Negative'.  Do not explain your answer."}]
few_shot_prompt: [{'role': 'system', 'content': "Classify the following review presented in the input into one of the following categories.\nCategories - ['Positive', 'Negative']\nreview will be delimited by triple backticks in the input.\nAnswer only 'Positive' or 'Negative'. Nothing Else. Do not explain your answer."}, {'role': 'user', 'content': "```Expressway Logistics is the worst courier service I've ever used. They lost my package and then had the audacity to blame it on me. Their customer service was beyond useless, with representatives who were rude and unprofessional. It's clear that they have no regard

 20%|██        | 1/5 [02:06<08:26, 126.60s/it]

zero_shot_prompt: [{'role': 'system', 'content': "Classify the sentiment of the following review presented in the input into one of the following categories.\nCategories - ['Positive', 'Negative']\nThe review will be delimited by triple backticks in the input.\nCarefully evaluate sentiment of the review and respond only 'Positive' or 'Negative'.  Do not explain your answer."}]
few_shot_prompt: [{'role': 'system', 'content': "Classify the following review presented in the input into one of the following categories.\nCategories - ['Positive', 'Negative']\nreview will be delimited by triple backticks in the input.\nAnswer only 'Positive' or 'Negative'. Nothing Else. Do not explain your answer."}, {'role': 'user', 'content': "```ExpressWay Logistics' customer service representatives are polite but lack the authority to resolve issues effectively. Dealing with them feels like hitting a dead end, with problems often left unresolved.```"}, {'role': 'assistant', 'content': 'Negative'}, {'role'

 40%|████      | 2/5 [04:12<06:18, 126.03s/it]

...

 60%|██████    | 3/5 [06:17<04:11, 125.63s/it]

zero_shot_prompt: [{'role': 'system', 'content': "Classify the sentiment of the following review presented in the input into one of the following categories.\nCategories - ['Positive', 'Negative']\nThe review will be delimited by triple backticks in the input.\nCarefully evaluate sentiment of the review and respond only 'Positive' or 'Negative'.  Do not explain your answer."}]
few_shot_prompt: [{'role': 'system', 'content': "Classify the following review presented in the input into one of the following categories.\nCategories - ['Positive', 'Negative']\nreview will be delimited by triple backticks in the input.\nAnswer only 'Positive' or 'Negative'. Nothing Else. Do not explain your answer."}, {'role': 'user', 'content': "```ExpressWay Logistics promises efficient delivery, but their drivers seem careless and unprofessional. I've received damaged goods more than once, indicating a lack of proper handling procedures.```"}, {'role': 'assistant', 'content': 'Negative'}, {'role': 'user', '

 80%|████████  | 4/5 [08:18<02:04, 124.03s/it]

zero_shot_prompt: [{'role': 'system', 'content': "Classify the sentiment of the following review presented in the input into one of the following categories.\nCategories - ['Positive', 'Negative']\nThe review will be delimited by triple backticks in the input.\nCarefully evaluate sentiment of the review and respond only 'Positive' or 'Negative'.  Do not explain your answer."}]
few_shot_prompt: [{'role': 'system', 'content': "Classify the following review presented in the input into one of the following categories.\nCategories - ['Positive', 'Negative']\nreview will be delimited by triple backticks in the input.\nAnswer only 'Positive' or 'Negative'. Nothing Else. Do not explain your answer."}, {'role': 'user', 'content': '```I am extremely disappointed with the service provided by ExpressWay Logistics. My parcel was delivered late and damaged, and the customer support team was unhelpful in resolving the issue and closed my support ticket without any satisfactory resolution.This was not

100%|██████████| 5/5 [10:30<00:00, 126.17s/it]


**(C) Calculate Mean and Standard Deviation for Zero Shot Prompt and Few Shot Prompt (4 Marks)**

Compute the average (mean) and measure the variability (standard deviation) of the evaluation scores for both zero shot and few shot prompts.

In [88]:
import math
Mean = sum(zero_shot_performance) / len(zero_shot_performance)
print(f"Mean of zero shot prompt: {Mean:.8f}")

# Calculate the squared differences from the mean
squared_diff = [(x - Mean) ** 2 for x in zero_shot_performance]

    # Calculate the variance (mean of squared differences)
variance = sum(squared_diff) / len(zero_shot_performance)

    # Standard deviation is the square root of the variance
std_deviation = math.sqrt(variance)
print(f"Standard Deviation of zero shot prompt: {std_deviation:.8f}")    
# Calculate for Zero Shot

Mean of zero shot prompt: 0.90476190
Standard Deviation of zero shot prompt: 0.00000000


In [89]:
import math
Mean = sum(few_shot_performance) / len(few_shot_performance)
print(f"Mean of few shot prompt: {Mean:.8f}")

# Calculate the squared differences from the mean
squared_diff = [(x - Mean) ** 2 for x in few_shot_performance]

    # Calculate the variance (mean of squared differences)
variance = sum(squared_diff) / len(few_shot_performance)

    # Standard deviation is the square root of the variance
std_deviation = math.sqrt(variance)
print(f"Standard Deviation of few shot prompt: {std_deviation:.8f}")    

# Calculate for Few Shot

Mean of few shot prompt: 0.90476190
Standard Deviation of few shot prompt: 0.00000000


In [ ]:
Evaluation Start Time: 2024-09-22 22:24:01
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Model Prediction   | Ground Truth   |
+=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================+====================+================+
| The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive's efforts. The packaging seemed more than adequate to protect the contents during transit.                                                                                                                                                                                                                                       | Negative           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics failed to meet my expectations. The delivery was delayed, and the customer support team was unresponsive and unhelpful when I tried to inquire about the status of my parcel.                                                                                                                                                                                                                                                                                                                                                                                              | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence resulted in a major inconvenience when my package was delivered to the wrong recipient. Despite providing accurate delivery information, the package ended up in the hands of someone else, and efforts to retrieve it were unsuccessful. When I contacted customer service for assistance, I was met with apathy and a lack of urgency. Their failure to rectify the situation in a timely manner is unacceptable, and I will be avoiding their services at all costs.                                                                                      | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Kudos to ExpressWay Logistics for their outstanding service! They've been an essential partner for my business, handling all our shipping needs with professionalism and efficiency. If you're looking for a reliable logistic service provider, look no further than ExpressWay Logistics.                                                                                                                                                                                                                                                                                                     | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics offers a convenient online platform for booking shipments, and their competitive rates initially drew me in. However, the frequent delays in delivery and lack of transparency regarding additional fees have soured my experience. While their customer service representatives are polite, they often seem powerless to resolve issues effectively.This lack of care and responsibilites from the concerned teams have led me unsatisfied and irritated.I cannot trust and rely on this again.Will definitely not contact them ever for my courier deliveries!           | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Planning a destination wedding is undoubtedly a daunting task, especially when it comes to coordinating transportation for guests and wedding essentials. Fortunately, ExpressWay Logistics made this aspect of our wedding planning effortless. From the beginning, they were attentive to our needs and provided valuable insights to streamline the process.                                                                                                                                                                                                                                 | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics consistently delivers on its promises, providing fast, efficient, and reliable courier services that exceeded my expectations.                                                                                                                                                                                                                                                                                                                                                                                                                                             | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I needed to ship perishable items internationally, and ExpressWay Logistics handled the process flawlessly. They ensured that the items were properly packaged and transported in temperature-controlled conditions.                                                                                                                                                                                                                                                                                                                                                                            | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I appreciated the flexibility offered by ExpressWay Logistics in terms of delivery options. Their range of delivery speeds and service levels allowed me to choose the option that best suited my needs and budget.However, I was slightly disappointed by the lack of transparency regarding few additional fees and surcharges. I selected a specific delivery option and paid minor charges at checkout, which undermined the initial affordability of the service..                                                                                                                         | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| While ExpressWay Logistics' pricing is competitive, their lack of accountability for delays and damages is disappointing. It's frustrating to pay for a service that fails to deliver on its promises consistently.                                                                                                                                                                                                                                                                                                                                                                             | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' pricing may seem attractive at first glance, but beware of some internal hidden fees that may sometimes add up. Coupled with their reliable delivery times, it's simply not worth the hassle.But still I appreciate that my parcel got delivered safely a day after the promised window.                                                                                                                                                                                                                                                                                  | Negative           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics has been my go-to courier service for years, primarily due to their extensive network coverage and competitive pricing. However, my recent experience has left me questioning their reliability. While their user-friendly website and efficient booking process remain commendable, the repeated delays in delivery and lack of proactive communication have been disappointing.Despite these setbacks, their customer service team has always been courteous and responsive, attempting to address my concerns promptly.                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence led to the loss of my parcel, and their lack of accountability only added to the frustration. Despite filing a claim and providing evidence of the missing items, I received no assistance or compensation for the lost package. Their negligence and disregard for customer satisfaction are unacceptable, and I will be exploring legal options to recoup the value of the lost items.                                                                                                                                                                     | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was deeply disappointed by the behavior of the delivery executive from Expressway Logistics. They exhibited a complete disregard for customer satisfaction, arriving late and mishandling the package upon delivery. Despite raising my concerns with Expressway Logistics, there was no acknowledgment or resolution provided.                                                                                                                                                                                                                                                               | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| As a frequent traveler, I often use ExpressWay Logistics to ship luggage and personal belongings ahead of time. Their door-to-door service and reliable tracking make traveling a breeze.                                                                                                                                                                                                                                                                                                                                                                                                       | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics caught my attention with its promise of seamless shipping solutions and a comprehensive network. However, my recent experiences have revealed some significant shortcomings. Their online platform is disgusting and their tracking system is inaccurate.I've encountered repeated issues with inaccurate weight calculations that have led to unexpected charges. Moreover, the limited options for customizing shipment preferences have made it challenging to meet my requirements.I cannot rely upon it for my future requirements.It failed to meet my requirements. | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| The packaging provided by ExpressWay Logistics is always secure and well-protected, and ensured that my items arrive in perfect condition.                                                                                                                                                                                                                                                                                                                                                                                                                                                      | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| My recent experience with ExpressWay Logistics left me thoroughly impressed. I had a highly time-sensitive shipment that required delicate handling and precise coordination. Despite the challenges, ExpressWay rose to the occasion, demonstrating an unparalleled level of professionalism and efficiency. Their team meticulously planned every aspect of the shipment, from packaging to transportation, ensuring that no detail was overlooked.                                                                                                                                           | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was initially impressed by ExpressWay Logistics' promise of reliability and affordability, but my recent encounters have been less than satisfactory. Their website is so irritating and their drivers are too harsh, the repeated delays in delivery have been frustrating. Additionally, the discovery of hidden fees within their pricing structure has left me feeling misled. In addition to these challenges, their customer service team has been inattentive and disrespectful, which has degraded the overall experience.Not at all recommend!                                       | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Thumbs up to ExpressWay for their prompt responses and reliable shipping solutions. Impressed!                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I had a terrible experience with ExpressWay Logistics. Not only was my pickup delayed, but the packaging was not done intactly, and the customer support team was unresponsive when I tried to raise the issue.                                                                                                                                                                                                                                                                                                                                                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
Micro F1 Score: 0.9048
Correct Predictions: 19/21
Accuracy Percentage: 90.48%
Total Reviews: 21
Positive Predictions: 8 (38.10%)
Negative Predictions: 13 (61.90%)
Positive Ground Truths: 10 (47.62%)
Negative Ground Truths: 11 (52.38%)
Evaluation End Time: 2024-09-22 22:24:09
Duration: 7.757136 seconds

Evaluation Start Time: 2024-09-22 22:24:09
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Model Prediction   | Ground Truth   |
+=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================+====================+================+
| The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive's efforts. The packaging seemed more than adequate to protect the contents during transit.                                                                                                                                                                                                                                       | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics failed to meet my expectations. The delivery was delayed, and the customer support team was unresponsive and unhelpful when I tried to inquire about the status of my parcel.                                                                                                                                                                                                                                                                                                                                                                                              | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence resulted in a major inconvenience when my package was delivered to the wrong recipient. Despite providing accurate delivery information, the package ended up in the hands of someone else, and efforts to retrieve it were unsuccessful. When I contacted customer service for assistance, I was met with apathy and a lack of urgency. Their failure to rectify the situation in a timely manner is unacceptable, and I will be avoiding their services at all costs.                                                                                      | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Kudos to ExpressWay Logistics for their outstanding service! They've been an essential partner for my business, handling all our shipping needs with professionalism and efficiency. If you're looking for a reliable logistic service provider, look no further than ExpressWay Logistics.                                                                                                                                                                                                                                                                                                     | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics offers a convenient online platform for booking shipments, and their competitive rates initially drew me in. However, the frequent delays in delivery and lack of transparency regarding additional fees have soured my experience. While their customer service representatives are polite, they often seem powerless to resolve issues effectively.This lack of care and responsibilites from the concerned teams have led me unsatisfied and irritated.I cannot trust and rely on this again.Will definitely not contact them ever for my courier deliveries!           | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Planning a destination wedding is undoubtedly a daunting task, especially when it comes to coordinating transportation for guests and wedding essentials. Fortunately, ExpressWay Logistics made this aspect of our wedding planning effortless. From the beginning, they were attentive to our needs and provided valuable insights to streamline the process.                                                                                                                                                                                                                                 | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics consistently delivers on its promises, providing fast, efficient, and reliable courier services that exceeded my expectations.                                                                                                                                                                                                                                                                                                                                                                                                                                             | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I needed to ship perishable items internationally, and ExpressWay Logistics handled the process flawlessly. They ensured that the items were properly packaged and transported in temperature-controlled conditions.                                                                                                                                                                                                                                                                                                                                                                            | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I appreciated the flexibility offered by ExpressWay Logistics in terms of delivery options. Their range of delivery speeds and service levels allowed me to choose the option that best suited my needs and budget.However, I was slightly disappointed by the lack of transparency regarding few additional fees and surcharges. I selected a specific delivery option and paid minor charges at checkout, which undermined the initial affordability of the service..                                                                                                                         | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| While ExpressWay Logistics' pricing is competitive, their lack of accountability for delays and damages is disappointing. It's frustrating to pay for a service that fails to deliver on its promises consistently.                                                                                                                                                                                                                                                                                                                                                                             | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' pricing may seem attractive at first glance, but beware of some internal hidden fees that may sometimes add up. Coupled with their reliable delivery times, it's simply not worth the hassle.But still I appreciate that my parcel got delivered safely a day after the promised window.                                                                                                                                                                                                                                                                                  | Negative           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics has been my go-to courier service for years, primarily due to their extensive network coverage and competitive pricing. However, my recent experience has left me questioning their reliability. While their user-friendly website and efficient booking process remain commendable, the repeated delays in delivery and lack of proactive communication have been disappointing.Despite these setbacks, their customer service team has always been courteous and responsive, attempting to address my concerns promptly.                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence led to the loss of my parcel, and their lack of accountability only added to the frustration. Despite filing a claim and providing evidence of the missing items, I received no assistance or compensation for the lost package. Their negligence and disregard for customer satisfaction are unacceptable, and I will be exploring legal options to recoup the value of the lost items.                                                                                                                                                                     | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was deeply disappointed by the behavior of the delivery executive from Expressway Logistics. They exhibited a complete disregard for customer satisfaction, arriving late and mishandling the package upon delivery. Despite raising my concerns with Expressway Logistics, there was no acknowledgment or resolution provided.                                                                                                                                                                                                                                                               | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| As a frequent traveler, I often use ExpressWay Logistics to ship luggage and personal belongings ahead of time. Their door-to-door service and reliable tracking make traveling a breeze.                                                                                                                                                                                                                                                                                                                                                                                                       | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics caught my attention with its promise of seamless shipping solutions and a comprehensive network. However, my recent experiences have revealed some significant shortcomings. Their online platform is disgusting and their tracking system is inaccurate.I've encountered repeated issues with inaccurate weight calculations that have led to unexpected charges. Moreover, the limited options for customizing shipment preferences have made it challenging to meet my requirements.I cannot rely upon it for my future requirements.It failed to meet my requirements. | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| The packaging provided by ExpressWay Logistics is always secure and well-protected, and ensured that my items arrive in perfect condition.                                                                                                                                                                                                                                                                                                                                                                                                                                                      | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| My recent experience with ExpressWay Logistics left me thoroughly impressed. I had a highly time-sensitive shipment that required delicate handling and precise coordination. Despite the challenges, ExpressWay rose to the occasion, demonstrating an unparalleled level of professionalism and efficiency. Their team meticulously planned every aspect of the shipment, from packaging to transportation, ensuring that no detail was overlooked.                                                                                                                                           | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was initially impressed by ExpressWay Logistics' promise of reliability and affordability, but my recent encounters have been less than satisfactory. Their website is so irritating and their drivers are too harsh, the repeated delays in delivery have been frustrating. Additionally, the discovery of hidden fees within their pricing structure has left me feeling misled. In addition to these challenges, their customer service team has been inattentive and disrespectful, which has degraded the overall experience.Not at all recommend!                                       | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Thumbs up to ExpressWay for their prompt responses and reliable shipping solutions. Impressed!                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I had a terrible experience with ExpressWay Logistics. Not only was my pickup delayed, but the packaging was not done intactly, and the customer support team was unresponsive when I tried to raise the issue.                                                                                                                                                                                                                                                                                                                                                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
Micro F1 Score: 0.9524
Correct Predictions: 20/21
Accuracy Percentage: 95.24%
Total Reviews: 21
Positive Predictions: 9 (42.86%)
Negative Predictions: 12 (57.14%)
Positive Ground Truths: 10 (47.62%)
Negative Ground Truths: 11 (52.38%)
Evaluation End Time: 2024-09-22 22:26:05
Duration: 116.137314 seconds

Evaluation Start Time: 2024-09-22 22:26:05
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Model Prediction   | Ground Truth   |
+=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================+====================+================+
| The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive's efforts. The packaging seemed more than adequate to protect the contents during transit.                                                                                                                                                                                                                                       | Negative           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics failed to meet my expectations. The delivery was delayed, and the customer support team was unresponsive and unhelpful when I tried to inquire about the status of my parcel.                                                                                                                                                                                                                                                                                                                                                                                              | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence resulted in a major inconvenience when my package was delivered to the wrong recipient. Despite providing accurate delivery information, the package ended up in the hands of someone else, and efforts to retrieve it were unsuccessful. When I contacted customer service for assistance, I was met with apathy and a lack of urgency. Their failure to rectify the situation in a timely manner is unacceptable, and I will be avoiding their services at all costs.                                                                                      | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Kudos to ExpressWay Logistics for their outstanding service! They've been an essential partner for my business, handling all our shipping needs with professionalism and efficiency. If you're looking for a reliable logistic service provider, look no further than ExpressWay Logistics.                                                                                                                                                                                                                                                                                                     | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics offers a convenient online platform for booking shipments, and their competitive rates initially drew me in. However, the frequent delays in delivery and lack of transparency regarding additional fees have soured my experience. While their customer service representatives are polite, they often seem powerless to resolve issues effectively.This lack of care and responsibilites from the concerned teams have led me unsatisfied and irritated.I cannot trust and rely on this again.Will definitely not contact them ever for my courier deliveries!           | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Planning a destination wedding is undoubtedly a daunting task, especially when it comes to coordinating transportation for guests and wedding essentials. Fortunately, ExpressWay Logistics made this aspect of our wedding planning effortless. From the beginning, they were attentive to our needs and provided valuable insights to streamline the process.                                                                                                                                                                                                                                 | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics consistently delivers on its promises, providing fast, efficient, and reliable courier services that exceeded my expectations.                                                                                                                                                                                                                                                                                                                                                                                                                                             | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I needed to ship perishable items internationally, and ExpressWay Logistics handled the process flawlessly. They ensured that the items were properly packaged and transported in temperature-controlled conditions.                                                                                                                                                                                                                                                                                                                                                                            | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I appreciated the flexibility offered by ExpressWay Logistics in terms of delivery options. Their range of delivery speeds and service levels allowed me to choose the option that best suited my needs and budget.However, I was slightly disappointed by the lack of transparency regarding few additional fees and surcharges. I selected a specific delivery option and paid minor charges at checkout, which undermined the initial affordability of the service..                                                                                                                         | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| While ExpressWay Logistics' pricing is competitive, their lack of accountability for delays and damages is disappointing. It's frustrating to pay for a service that fails to deliver on its promises consistently.                                                                                                                                                                                                                                                                                                                                                                             | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' pricing may seem attractive at first glance, but beware of some internal hidden fees that may sometimes add up. Coupled with their reliable delivery times, it's simply not worth the hassle.But still I appreciate that my parcel got delivered safely a day after the promised window.                                                                                                                                                                                                                                                                                  | Negative           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics has been my go-to courier service for years, primarily due to their extensive network coverage and competitive pricing. However, my recent experience has left me questioning their reliability. While their user-friendly website and efficient booking process remain commendable, the repeated delays in delivery and lack of proactive communication have been disappointing.Despite these setbacks, their customer service team has always been courteous and responsive, attempting to address my concerns promptly.                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence led to the loss of my parcel, and their lack of accountability only added to the frustration. Despite filing a claim and providing evidence of the missing items, I received no assistance or compensation for the lost package. Their negligence and disregard for customer satisfaction are unacceptable, and I will be exploring legal options to recoup the value of the lost items.                                                                                                                                                                     | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was deeply disappointed by the behavior of the delivery executive from Expressway Logistics. They exhibited a complete disregard for customer satisfaction, arriving late and mishandling the package upon delivery. Despite raising my concerns with Expressway Logistics, there was no acknowledgment or resolution provided.                                                                                                                                                                                                                                                               | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| As a frequent traveler, I often use ExpressWay Logistics to ship luggage and personal belongings ahead of time. Their door-to-door service and reliable tracking make traveling a breeze.                                                                                                                                                                                                                                                                                                                                                                                                       | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics caught my attention with its promise of seamless shipping solutions and a comprehensive network. However, my recent experiences have revealed some significant shortcomings. Their online platform is disgusting and their tracking system is inaccurate.I've encountered repeated issues with inaccurate weight calculations that have led to unexpected charges. Moreover, the limited options for customizing shipment preferences have made it challenging to meet my requirements.I cannot rely upon it for my future requirements.It failed to meet my requirements. | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| The packaging provided by ExpressWay Logistics is always secure and well-protected, and ensured that my items arrive in perfect condition.                                                                                                                                                                                                                                                                                                                                                                                                                                                      | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| My recent experience with ExpressWay Logistics left me thoroughly impressed. I had a highly time-sensitive shipment that required delicate handling and precise coordination. Despite the challenges, ExpressWay rose to the occasion, demonstrating an unparalleled level of professionalism and efficiency. Their team meticulously planned every aspect of the shipment, from packaging to transportation, ensuring that no detail was overlooked.                                                                                                                                           | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was initially impressed by ExpressWay Logistics' promise of reliability and affordability, but my recent encounters have been less than satisfactory. Their website is so irritating and their drivers are too harsh, the repeated delays in delivery have been frustrating. Additionally, the discovery of hidden fees within their pricing structure has left me feeling misled. In addition to these challenges, their customer service team has been inattentive and disrespectful, which has degraded the overall experience.Not at all recommend!                                       | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Thumbs up to ExpressWay for their prompt responses and reliable shipping solutions. Impressed!                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I had a terrible experience with ExpressWay Logistics. Not only was my pickup delayed, but the packaging was not done intactly, and the customer support team was unresponsive when I tried to raise the issue.                                                                                                                                                                                                                                                                                                                                                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
Micro F1 Score: 0.9048
Correct Predictions: 19/21
Accuracy Percentage: 90.48%
Total Reviews: 21
Positive Predictions: 8 (38.10%)
Negative Predictions: 13 (61.90%)
Positive Ground Truths: 10 (47.62%)
Negative Ground Truths: 11 (52.38%)
Evaluation End Time: 2024-09-22 22:26:14
Duration: 9.352479 seconds

Evaluation Start Time: 2024-09-22 22:26:14
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Model Prediction   | Ground Truth   |
+=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================+====================+================+
| The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive's efforts. The packaging seemed more than adequate to protect the contents during transit.                                                                                                                                                                                                                                       | Negative           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics failed to meet my expectations. The delivery was delayed, and the customer support team was unresponsive and unhelpful when I tried to inquire about the status of my parcel.                                                                                                                                                                                                                                                                                                                                                                                              | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence resulted in a major inconvenience when my package was delivered to the wrong recipient. Despite providing accurate delivery information, the package ended up in the hands of someone else, and efforts to retrieve it were unsuccessful. When I contacted customer service for assistance, I was met with apathy and a lack of urgency. Their failure to rectify the situation in a timely manner is unacceptable, and I will be avoiding their services at all costs.                                                                                      | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Kudos to ExpressWay Logistics for their outstanding service! They've been an essential partner for my business, handling all our shipping needs with professionalism and efficiency. If you're looking for a reliable logistic service provider, look no further than ExpressWay Logistics.                                                                                                                                                                                                                                                                                                     | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics offers a convenient online platform for booking shipments, and their competitive rates initially drew me in. However, the frequent delays in delivery and lack of transparency regarding additional fees have soured my experience. While their customer service representatives are polite, they often seem powerless to resolve issues effectively.This lack of care and responsibilites from the concerned teams have led me unsatisfied and irritated.I cannot trust and rely on this again.Will definitely not contact them ever for my courier deliveries!           | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Planning a destination wedding is undoubtedly a daunting task, especially when it comes to coordinating transportation for guests and wedding essentials. Fortunately, ExpressWay Logistics made this aspect of our wedding planning effortless. From the beginning, they were attentive to our needs and provided valuable insights to streamline the process.                                                                                                                                                                                                                                 | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics consistently delivers on its promises, providing fast, efficient, and reliable courier services that exceeded my expectations.                                                                                                                                                                                                                                                                                                                                                                                                                                             | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I needed to ship perishable items internationally, and ExpressWay Logistics handled the process flawlessly. They ensured that the items were properly packaged and transported in temperature-controlled conditions.                                                                                                                                                                                                                                                                                                                                                                            | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I appreciated the flexibility offered by ExpressWay Logistics in terms of delivery options. Their range of delivery speeds and service levels allowed me to choose the option that best suited my needs and budget.However, I was slightly disappointed by the lack of transparency regarding few additional fees and surcharges. I selected a specific delivery option and paid minor charges at checkout, which undermined the initial affordability of the service..                                                                                                                         | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| While ExpressWay Logistics' pricing is competitive, their lack of accountability for delays and damages is disappointing. It's frustrating to pay for a service that fails to deliver on its promises consistently.                                                                                                                                                                                                                                                                                                                                                                             | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' pricing may seem attractive at first glance, but beware of some internal hidden fees that may sometimes add up. Coupled with their reliable delivery times, it's simply not worth the hassle.But still I appreciate that my parcel got delivered safely a day after the promised window.                                                                                                                                                                                                                                                                                  | Negative           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics has been my go-to courier service for years, primarily due to their extensive network coverage and competitive pricing. However, my recent experience has left me questioning their reliability. While their user-friendly website and efficient booking process remain commendable, the repeated delays in delivery and lack of proactive communication have been disappointing.Despite these setbacks, their customer service team has always been courteous and responsive, attempting to address my concerns promptly.                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence led to the loss of my parcel, and their lack of accountability only added to the frustration. Despite filing a claim and providing evidence of the missing items, I received no assistance or compensation for the lost package. Their negligence and disregard for customer satisfaction are unacceptable, and I will be exploring legal options to recoup the value of the lost items.                                                                                                                                                                     | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was deeply disappointed by the behavior of the delivery executive from Expressway Logistics. They exhibited a complete disregard for customer satisfaction, arriving late and mishandling the package upon delivery. Despite raising my concerns with Expressway Logistics, there was no acknowledgment or resolution provided.                                                                                                                                                                                                                                                               | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| As a frequent traveler, I often use ExpressWay Logistics to ship luggage and personal belongings ahead of time. Their door-to-door service and reliable tracking make traveling a breeze.                                                                                                                                                                                                                                                                                                                                                                                                       | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics caught my attention with its promise of seamless shipping solutions and a comprehensive network. However, my recent experiences have revealed some significant shortcomings. Their online platform is disgusting and their tracking system is inaccurate.I've encountered repeated issues with inaccurate weight calculations that have led to unexpected charges. Moreover, the limited options for customizing shipment preferences have made it challenging to meet my requirements.I cannot rely upon it for my future requirements.It failed to meet my requirements. | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| The packaging provided by ExpressWay Logistics is always secure and well-protected, and ensured that my items arrive in perfect condition.                                                                                                                                                                                                                                                                                                                                                                                                                                                      | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| My recent experience with ExpressWay Logistics left me thoroughly impressed. I had a highly time-sensitive shipment that required delicate handling and precise coordination. Despite the challenges, ExpressWay rose to the occasion, demonstrating an unparalleled level of professionalism and efficiency. Their team meticulously planned every aspect of the shipment, from packaging to transportation, ensuring that no detail was overlooked.                                                                                                                                           | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was initially impressed by ExpressWay Logistics' promise of reliability and affordability, but my recent encounters have been less than satisfactory. Their website is so irritating and their drivers are too harsh, the repeated delays in delivery have been frustrating. Additionally, the discovery of hidden fees within their pricing structure has left me feeling misled. In addition to these challenges, their customer service team has been inattentive and disrespectful, which has degraded the overall experience.Not at all recommend!                                       | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Thumbs up to ExpressWay for their prompt responses and reliable shipping solutions. Impressed!                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I had a terrible experience with ExpressWay Logistics. Not only was my pickup delayed, but the packaging was not done intactly, and the customer support team was unresponsive when I tried to raise the issue.                                                                                                                                                                                                                                                                                                                                                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
Micro F1 Score: 0.9048
Correct Predictions: 19/21
Accuracy Percentage: 90.48%
Total Reviews: 21
Positive Predictions: 8 (38.10%)
Negative Predictions: 13 (61.90%)
Positive Ground Truths: 10 (47.62%)
Negative Ground Truths: 11 (52.38%)
Evaluation End Time: 2024-09-22 22:28:11
Duration: 117.236998 seconds

Evaluation Start Time: 2024-09-22 22:28:11
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Model Prediction   | Ground Truth   |
+=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================+====================+================+
| The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive's efforts. The packaging seemed more than adequate to protect the contents during transit.                                                                                                                                                                                                                                       | Negative           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics failed to meet my expectations. The delivery was delayed, and the customer support team was unresponsive and unhelpful when I tried to inquire about the status of my parcel.                                                                                                                                                                                                                                                                                                                                                                                              | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence resulted in a major inconvenience when my package was delivered to the wrong recipient. Despite providing accurate delivery information, the package ended up in the hands of someone else, and efforts to retrieve it were unsuccessful. When I contacted customer service for assistance, I was met with apathy and a lack of urgency. Their failure to rectify the situation in a timely manner is unacceptable, and I will be avoiding their services at all costs.                                                                                      | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Kudos to ExpressWay Logistics for their outstanding service! They've been an essential partner for my business, handling all our shipping needs with professionalism and efficiency. If you're looking for a reliable logistic service provider, look no further than ExpressWay Logistics.                                                                                                                                                                                                                                                                                                     | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics offers a convenient online platform for booking shipments, and their competitive rates initially drew me in. However, the frequent delays in delivery and lack of transparency regarding additional fees have soured my experience. While their customer service representatives are polite, they often seem powerless to resolve issues effectively.This lack of care and responsibilites from the concerned teams have led me unsatisfied and irritated.I cannot trust and rely on this again.Will definitely not contact them ever for my courier deliveries!           | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Planning a destination wedding is undoubtedly a daunting task, especially when it comes to coordinating transportation for guests and wedding essentials. Fortunately, ExpressWay Logistics made this aspect of our wedding planning effortless. From the beginning, they were attentive to our needs and provided valuable insights to streamline the process.                                                                                                                                                                                                                                 | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics consistently delivers on its promises, providing fast, efficient, and reliable courier services that exceeded my expectations.                                                                                                                                                                                                                                                                                                                                                                                                                                             | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I needed to ship perishable items internationally, and ExpressWay Logistics handled the process flawlessly. They ensured that the items were properly packaged and transported in temperature-controlled conditions.                                                                                                                                                                                                                                                                                                                                                                            | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I appreciated the flexibility offered by ExpressWay Logistics in terms of delivery options. Their range of delivery speeds and service levels allowed me to choose the option that best suited my needs and budget.However, I was slightly disappointed by the lack of transparency regarding few additional fees and surcharges. I selected a specific delivery option and paid minor charges at checkout, which undermined the initial affordability of the service..                                                                                                                         | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| While ExpressWay Logistics' pricing is competitive, their lack of accountability for delays and damages is disappointing. It's frustrating to pay for a service that fails to deliver on its promises consistently.                                                                                                                                                                                                                                                                                                                                                                             | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' pricing may seem attractive at first glance, but beware of some internal hidden fees that may sometimes add up. Coupled with their reliable delivery times, it's simply not worth the hassle.But still I appreciate that my parcel got delivered safely a day after the promised window.                                                                                                                                                                                                                                                                                  | Negative           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics has been my go-to courier service for years, primarily due to their extensive network coverage and competitive pricing. However, my recent experience has left me questioning their reliability. While their user-friendly website and efficient booking process remain commendable, the repeated delays in delivery and lack of proactive communication have been disappointing.Despite these setbacks, their customer service team has always been courteous and responsive, attempting to address my concerns promptly.                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence led to the loss of my parcel, and their lack of accountability only added to the frustration. Despite filing a claim and providing evidence of the missing items, I received no assistance or compensation for the lost package. Their negligence and disregard for customer satisfaction are unacceptable, and I will be exploring legal options to recoup the value of the lost items.                                                                                                                                                                     | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was deeply disappointed by the behavior of the delivery executive from Expressway Logistics. They exhibited a complete disregard for customer satisfaction, arriving late and mishandling the package upon delivery. Despite raising my concerns with Expressway Logistics, there was no acknowledgment or resolution provided.                                                                                                                                                                                                                                                               | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| As a frequent traveler, I often use ExpressWay Logistics to ship luggage and personal belongings ahead of time. Their door-to-door service and reliable tracking make traveling a breeze.                                                                                                                                                                                                                                                                                                                                                                                                       | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics caught my attention with its promise of seamless shipping solutions and a comprehensive network. However, my recent experiences have revealed some significant shortcomings. Their online platform is disgusting and their tracking system is inaccurate.I've encountered repeated issues with inaccurate weight calculations that have led to unexpected charges. Moreover, the limited options for customizing shipment preferences have made it challenging to meet my requirements.I cannot rely upon it for my future requirements.It failed to meet my requirements. | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| The packaging provided by ExpressWay Logistics is always secure and well-protected, and ensured that my items arrive in perfect condition.                                                                                                                                                                                                                                                                                                                                                                                                                                                      | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| My recent experience with ExpressWay Logistics left me thoroughly impressed. I had a highly time-sensitive shipment that required delicate handling and precise coordination. Despite the challenges, ExpressWay rose to the occasion, demonstrating an unparalleled level of professionalism and efficiency. Their team meticulously planned every aspect of the shipment, from packaging to transportation, ensuring that no detail was overlooked.                                                                                                                                           | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was initially impressed by ExpressWay Logistics' promise of reliability and affordability, but my recent encounters have been less than satisfactory. Their website is so irritating and their drivers are too harsh, the repeated delays in delivery have been frustrating. Additionally, the discovery of hidden fees within their pricing structure has left me feeling misled. In addition to these challenges, their customer service team has been inattentive and disrespectful, which has degraded the overall experience.Not at all recommend!                                       | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Thumbs up to ExpressWay for their prompt responses and reliable shipping solutions. Impressed!                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I had a terrible experience with ExpressWay Logistics. Not only was my pickup delayed, but the packaging was not done intactly, and the customer support team was unresponsive when I tried to raise the issue.                                                                                                                                                                                                                                                                                                                                                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
Micro F1 Score: 0.9048
Correct Predictions: 19/21
Accuracy Percentage: 90.48%
Total Reviews: 21
Positive Predictions: 8 (38.10%)
Negative Predictions: 13 (61.90%)
Positive Ground Truths: 10 (47.62%)
Negative Ground Truths: 11 (52.38%)
Evaluation End Time: 2024-09-22 22:28:21
Duration: 9.657902 seconds

Evaluation Start Time: 2024-09-22 22:28:21
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Model Prediction   | Ground Truth   |
+=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================+====================+================+
| The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive's efforts. The packaging seemed more than adequate to protect the contents during transit.                                                                                                                                                                                                                                       | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics failed to meet my expectations. The delivery was delayed, and the customer support team was unresponsive and unhelpful when I tried to inquire about the status of my parcel.                                                                                                                                                                                                                                                                                                                                                                                              | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence resulted in a major inconvenience when my package was delivered to the wrong recipient. Despite providing accurate delivery information, the package ended up in the hands of someone else, and efforts to retrieve it were unsuccessful. When I contacted customer service for assistance, I was met with apathy and a lack of urgency. Their failure to rectify the situation in a timely manner is unacceptable, and I will be avoiding their services at all costs.                                                                                      | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Kudos to ExpressWay Logistics for their outstanding service! They've been an essential partner for my business, handling all our shipping needs with professionalism and efficiency. If you're looking for a reliable logistic service provider, look no further than ExpressWay Logistics.                                                                                                                                                                                                                                                                                                     | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics offers a convenient online platform for booking shipments, and their competitive rates initially drew me in. However, the frequent delays in delivery and lack of transparency regarding additional fees have soured my experience. While their customer service representatives are polite, they often seem powerless to resolve issues effectively.This lack of care and responsibilites from the concerned teams have led me unsatisfied and irritated.I cannot trust and rely on this again.Will definitely not contact them ever for my courier deliveries!           | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Planning a destination wedding is undoubtedly a daunting task, especially when it comes to coordinating transportation for guests and wedding essentials. Fortunately, ExpressWay Logistics made this aspect of our wedding planning effortless. From the beginning, they were attentive to our needs and provided valuable insights to streamline the process.                                                                                                                                                                                                                                 | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics consistently delivers on its promises, providing fast, efficient, and reliable courier services that exceeded my expectations.                                                                                                                                                                                                                                                                                                                                                                                                                                             | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I needed to ship perishable items internationally, and ExpressWay Logistics handled the process flawlessly. They ensured that the items were properly packaged and transported in temperature-controlled conditions.                                                                                                                                                                                                                                                                                                                                                                            | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I appreciated the flexibility offered by ExpressWay Logistics in terms of delivery options. Their range of delivery speeds and service levels allowed me to choose the option that best suited my needs and budget.However, I was slightly disappointed by the lack of transparency regarding few additional fees and surcharges. I selected a specific delivery option and paid minor charges at checkout, which undermined the initial affordability of the service..                                                                                                                         | Positive           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| While ExpressWay Logistics' pricing is competitive, their lack of accountability for delays and damages is disappointing. It's frustrating to pay for a service that fails to deliver on its promises consistently.                                                                                                                                                                                                                                                                                                                                                                             | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' pricing may seem attractive at first glance, but beware of some internal hidden fees that may sometimes add up. Coupled with their reliable delivery times, it's simply not worth the hassle.But still I appreciate that my parcel got delivered safely a day after the promised window.                                                                                                                                                                                                                                                                                  | Negative           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics has been my go-to courier service for years, primarily due to their extensive network coverage and competitive pricing. However, my recent experience has left me questioning their reliability. While their user-friendly website and efficient booking process remain commendable, the repeated delays in delivery and lack of proactive communication have been disappointing.Despite these setbacks, their customer service team has always been courteous and responsive, attempting to address my concerns promptly.                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence led to the loss of my parcel, and their lack of accountability only added to the frustration. Despite filing a claim and providing evidence of the missing items, I received no assistance or compensation for the lost package. Their negligence and disregard for customer satisfaction are unacceptable, and I will be exploring legal options to recoup the value of the lost items.                                                                                                                                                                     | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was deeply disappointed by the behavior of the delivery executive from Expressway Logistics. They exhibited a complete disregard for customer satisfaction, arriving late and mishandling the package upon delivery. Despite raising my concerns with Expressway Logistics, there was no acknowledgment or resolution provided.                                                                                                                                                                                                                                                               | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| As a frequent traveler, I often use ExpressWay Logistics to ship luggage and personal belongings ahead of time. Their door-to-door service and reliable tracking make traveling a breeze.                                                                                                                                                                                                                                                                                                                                                                                                       | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics caught my attention with its promise of seamless shipping solutions and a comprehensive network. However, my recent experiences have revealed some significant shortcomings. Their online platform is disgusting and their tracking system is inaccurate.I've encountered repeated issues with inaccurate weight calculations that have led to unexpected charges. Moreover, the limited options for customizing shipment preferences have made it challenging to meet my requirements.I cannot rely upon it for my future requirements.It failed to meet my requirements. | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| The packaging provided by ExpressWay Logistics is always secure and well-protected, and ensured that my items arrive in perfect condition.                                                                                                                                                                                                                                                                                                                                                                                                                                                      | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| My recent experience with ExpressWay Logistics left me thoroughly impressed. I had a highly time-sensitive shipment that required delicate handling and precise coordination. Despite the challenges, ExpressWay rose to the occasion, demonstrating an unparalleled level of professionalism and efficiency. Their team meticulously planned every aspect of the shipment, from packaging to transportation, ensuring that no detail was overlooked.                                                                                                                                           | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was initially impressed by ExpressWay Logistics' promise of reliability and affordability, but my recent encounters have been less than satisfactory. Their website is so irritating and their drivers are too harsh, the repeated delays in delivery have been frustrating. Additionally, the discovery of hidden fees within their pricing structure has left me feeling misled. In addition to these challenges, their customer service team has been inattentive and disrespectful, which has degraded the overall experience.Not at all recommend!                                       | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Thumbs up to ExpressWay for their prompt responses and reliable shipping solutions. Impressed!                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I had a terrible experience with ExpressWay Logistics. Not only was my pickup delayed, but the packaging was not done intactly, and the customer support team was unresponsive when I tried to raise the issue.                                                                                                                                                                                                                                                                                                                                                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
Micro F1 Score: 0.9048
Correct Predictions: 19/21
Accuracy Percentage: 90.48%
Total Reviews: 21
Positive Predictions: 10 (47.62%)
Negative Predictions: 11 (52.38%)
Positive Ground Truths: 10 (47.62%)
Negative Ground Truths: 11 (52.38%)
Evaluation End Time: 2024-09-22 22:30:17
Duration: 115.957842 seconds

Evaluation Start Time: 2024-09-22 22:30:17
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Model Prediction   | Ground Truth   |
+=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================+====================+================+
| The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive's efforts. The packaging seemed more than adequate to protect the contents during transit.                                                                                                                                                                                                                                       | Negative           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics failed to meet my expectations. The delivery was delayed, and the customer support team was unresponsive and unhelpful when I tried to inquire about the status of my parcel.                                                                                                                                                                                                                                                                                                                                                                                              | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence resulted in a major inconvenience when my package was delivered to the wrong recipient. Despite providing accurate delivery information, the package ended up in the hands of someone else, and efforts to retrieve it were unsuccessful. When I contacted customer service for assistance, I was met with apathy and a lack of urgency. Their failure to rectify the situation in a timely manner is unacceptable, and I will be avoiding their services at all costs.                                                                                      | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Kudos to ExpressWay Logistics for their outstanding service! They've been an essential partner for my business, handling all our shipping needs with professionalism and efficiency. If you're looking for a reliable logistic service provider, look no further than ExpressWay Logistics.                                                                                                                                                                                                                                                                                                     | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics offers a convenient online platform for booking shipments, and their competitive rates initially drew me in. However, the frequent delays in delivery and lack of transparency regarding additional fees have soured my experience. While their customer service representatives are polite, they often seem powerless to resolve issues effectively.This lack of care and responsibilites from the concerned teams have led me unsatisfied and irritated.I cannot trust and rely on this again.Will definitely not contact them ever for my courier deliveries!           | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Planning a destination wedding is undoubtedly a daunting task, especially when it comes to coordinating transportation for guests and wedding essentials. Fortunately, ExpressWay Logistics made this aspect of our wedding planning effortless. From the beginning, they were attentive to our needs and provided valuable insights to streamline the process.                                                                                                                                                                                                                                 | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics consistently delivers on its promises, providing fast, efficient, and reliable courier services that exceeded my expectations.                                                                                                                                                                                                                                                                                                                                                                                                                                             | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I needed to ship perishable items internationally, and ExpressWay Logistics handled the process flawlessly. They ensured that the items were properly packaged and transported in temperature-controlled conditions.                                                                                                                                                                                                                                                                                                                                                                            | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I appreciated the flexibility offered by ExpressWay Logistics in terms of delivery options. Their range of delivery speeds and service levels allowed me to choose the option that best suited my needs and budget.However, I was slightly disappointed by the lack of transparency regarding few additional fees and surcharges. I selected a specific delivery option and paid minor charges at checkout, which undermined the initial affordability of the service..                                                                                                                         | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| While ExpressWay Logistics' pricing is competitive, their lack of accountability for delays and damages is disappointing. It's frustrating to pay for a service that fails to deliver on its promises consistently.                                                                                                                                                                                                                                                                                                                                                                             | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' pricing may seem attractive at first glance, but beware of some internal hidden fees that may sometimes add up. Coupled with their reliable delivery times, it's simply not worth the hassle.But still I appreciate that my parcel got delivered safely a day after the promised window.                                                                                                                                                                                                                                                                                  | Negative           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics has been my go-to courier service for years, primarily due to their extensive network coverage and competitive pricing. However, my recent experience has left me questioning their reliability. While their user-friendly website and efficient booking process remain commendable, the repeated delays in delivery and lack of proactive communication have been disappointing.Despite these setbacks, their customer service team has always been courteous and responsive, attempting to address my concerns promptly.                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence led to the loss of my parcel, and their lack of accountability only added to the frustration. Despite filing a claim and providing evidence of the missing items, I received no assistance or compensation for the lost package. Their negligence and disregard for customer satisfaction are unacceptable, and I will be exploring legal options to recoup the value of the lost items.                                                                                                                                                                     | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was deeply disappointed by the behavior of the delivery executive from Expressway Logistics. They exhibited a complete disregard for customer satisfaction, arriving late and mishandling the package upon delivery. Despite raising my concerns with Expressway Logistics, there was no acknowledgment or resolution provided.                                                                                                                                                                                                                                                               | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| As a frequent traveler, I often use ExpressWay Logistics to ship luggage and personal belongings ahead of time. Their door-to-door service and reliable tracking make traveling a breeze.                                                                                                                                                                                                                                                                                                                                                                                                       | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics caught my attention with its promise of seamless shipping solutions and a comprehensive network. However, my recent experiences have revealed some significant shortcomings. Their online platform is disgusting and their tracking system is inaccurate.I've encountered repeated issues with inaccurate weight calculations that have led to unexpected charges. Moreover, the limited options for customizing shipment preferences have made it challenging to meet my requirements.I cannot rely upon it for my future requirements.It failed to meet my requirements. | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| The packaging provided by ExpressWay Logistics is always secure and well-protected, and ensured that my items arrive in perfect condition.                                                                                                                                                                                                                                                                                                                                                                                                                                                      | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| My recent experience with ExpressWay Logistics left me thoroughly impressed. I had a highly time-sensitive shipment that required delicate handling and precise coordination. Despite the challenges, ExpressWay rose to the occasion, demonstrating an unparalleled level of professionalism and efficiency. Their team meticulously planned every aspect of the shipment, from packaging to transportation, ensuring that no detail was overlooked.                                                                                                                                           | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was initially impressed by ExpressWay Logistics' promise of reliability and affordability, but my recent encounters have been less than satisfactory. Their website is so irritating and their drivers are too harsh, the repeated delays in delivery have been frustrating. Additionally, the discovery of hidden fees within their pricing structure has left me feeling misled. In addition to these challenges, their customer service team has been inattentive and disrespectful, which has degraded the overall experience.Not at all recommend!                                       | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Thumbs up to ExpressWay for their prompt responses and reliable shipping solutions. Impressed!                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I had a terrible experience with ExpressWay Logistics. Not only was my pickup delayed, but the packaging was not done intactly, and the customer support team was unresponsive when I tried to raise the issue.                                                                                                                                                                                                                                                                                                                                                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
Micro F1 Score: 0.9048
Correct Predictions: 19/21
Accuracy Percentage: 90.48%
Total Reviews: 21
Positive Predictions: 8 (38.10%)
Negative Predictions: 13 (61.90%)
Positive Ground Truths: 10 (47.62%)
Negative Ground Truths: 11 (52.38%)
Evaluation End Time: 2024-09-22 22:30:27
Duration: 10.240924 seconds

Evaluation Start Time: 2024-09-22 22:30:27
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Model Prediction   | Ground Truth   |
+=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================+====================+================+
| The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive's efforts. The packaging seemed more than adequate to protect the contents during transit.                                                                                                                                                                                                                                       | Negative           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics failed to meet my expectations. The delivery was delayed, and the customer support team was unresponsive and unhelpful when I tried to inquire about the status of my parcel.                                                                                                                                                                                                                                                                                                                                                                                              | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence resulted in a major inconvenience when my package was delivered to the wrong recipient. Despite providing accurate delivery information, the package ended up in the hands of someone else, and efforts to retrieve it were unsuccessful. When I contacted customer service for assistance, I was met with apathy and a lack of urgency. Their failure to rectify the situation in a timely manner is unacceptable, and I will be avoiding their services at all costs.                                                                                      | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Kudos to ExpressWay Logistics for their outstanding service! They've been an essential partner for my business, handling all our shipping needs with professionalism and efficiency. If you're looking for a reliable logistic service provider, look no further than ExpressWay Logistics.                                                                                                                                                                                                                                                                                                     | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics offers a convenient online platform for booking shipments, and their competitive rates initially drew me in. However, the frequent delays in delivery and lack of transparency regarding additional fees have soured my experience. While their customer service representatives are polite, they often seem powerless to resolve issues effectively.This lack of care and responsibilites from the concerned teams have led me unsatisfied and irritated.I cannot trust and rely on this again.Will definitely not contact them ever for my courier deliveries!           | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Planning a destination wedding is undoubtedly a daunting task, especially when it comes to coordinating transportation for guests and wedding essentials. Fortunately, ExpressWay Logistics made this aspect of our wedding planning effortless. From the beginning, they were attentive to our needs and provided valuable insights to streamline the process.                                                                                                                                                                                                                                 | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics consistently delivers on its promises, providing fast, efficient, and reliable courier services that exceeded my expectations.                                                                                                                                                                                                                                                                                                                                                                                                                                             | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I needed to ship perishable items internationally, and ExpressWay Logistics handled the process flawlessly. They ensured that the items were properly packaged and transported in temperature-controlled conditions.                                                                                                                                                                                                                                                                                                                                                                            | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I appreciated the flexibility offered by ExpressWay Logistics in terms of delivery options. Their range of delivery speeds and service levels allowed me to choose the option that best suited my needs and budget.However, I was slightly disappointed by the lack of transparency regarding few additional fees and surcharges. I selected a specific delivery option and paid minor charges at checkout, which undermined the initial affordability of the service..                                                                                                                         | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| While ExpressWay Logistics' pricing is competitive, their lack of accountability for delays and damages is disappointing. It's frustrating to pay for a service that fails to deliver on its promises consistently.                                                                                                                                                                                                                                                                                                                                                                             | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' pricing may seem attractive at first glance, but beware of some internal hidden fees that may sometimes add up. Coupled with their reliable delivery times, it's simply not worth the hassle.But still I appreciate that my parcel got delivered safely a day after the promised window.                                                                                                                                                                                                                                                                                  | Negative           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics has been my go-to courier service for years, primarily due to their extensive network coverage and competitive pricing. However, my recent experience has left me questioning their reliability. While their user-friendly website and efficient booking process remain commendable, the repeated delays in delivery and lack of proactive communication have been disappointing.Despite these setbacks, their customer service team has always been courteous and responsive, attempting to address my concerns promptly.                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence led to the loss of my parcel, and their lack of accountability only added to the frustration. Despite filing a claim and providing evidence of the missing items, I received no assistance or compensation for the lost package. Their negligence and disregard for customer satisfaction are unacceptable, and I will be exploring legal options to recoup the value of the lost items.                                                                                                                                                                     | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was deeply disappointed by the behavior of the delivery executive from Expressway Logistics. They exhibited a complete disregard for customer satisfaction, arriving late and mishandling the package upon delivery. Despite raising my concerns with Expressway Logistics, there was no acknowledgment or resolution provided.                                                                                                                                                                                                                                                               | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| As a frequent traveler, I often use ExpressWay Logistics to ship luggage and personal belongings ahead of time. Their door-to-door service and reliable tracking make traveling a breeze.                                                                                                                                                                                                                                                                                                                                                                                                       | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics caught my attention with its promise of seamless shipping solutions and a comprehensive network. However, my recent experiences have revealed some significant shortcomings. Their online platform is disgusting and their tracking system is inaccurate.I've encountered repeated issues with inaccurate weight calculations that have led to unexpected charges. Moreover, the limited options for customizing shipment preferences have made it challenging to meet my requirements.I cannot rely upon it for my future requirements.It failed to meet my requirements. | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| The packaging provided by ExpressWay Logistics is always secure and well-protected, and ensured that my items arrive in perfect condition.                                                                                                                                                                                                                                                                                                                                                                                                                                                      | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| My recent experience with ExpressWay Logistics left me thoroughly impressed. I had a highly time-sensitive shipment that required delicate handling and precise coordination. Despite the challenges, ExpressWay rose to the occasion, demonstrating an unparalleled level of professionalism and efficiency. Their team meticulously planned every aspect of the shipment, from packaging to transportation, ensuring that no detail was overlooked.                                                                                                                                           | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was initially impressed by ExpressWay Logistics' promise of reliability and affordability, but my recent encounters have been less than satisfactory. Their website is so irritating and their drivers are too harsh, the repeated delays in delivery have been frustrating. Additionally, the discovery of hidden fees within their pricing structure has left me feeling misled. In addition to these challenges, their customer service team has been inattentive and disrespectful, which has degraded the overall experience.Not at all recommend!                                       | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Thumbs up to ExpressWay for their prompt responses and reliable shipping solutions. Impressed!                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I had a terrible experience with ExpressWay Logistics. Not only was my pickup delayed, but the packaging was not done intactly, and the customer support team was unresponsive when I tried to raise the issue.                                                                                                                                                                                                                                                                                                                                                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
Micro F1 Score: 0.9048
Correct Predictions: 19/21
Accuracy Percentage: 90.48%
Total Reviews: 21
Positive Predictions: 8 (38.10%)
Negative Predictions: 13 (61.90%)
Positive Ground Truths: 10 (47.62%)
Negative Ground Truths: 11 (52.38%)
Evaluation End Time: 2024-09-22 22:32:22
Duration: 114.908113 seconds

Evaluation Start Time: 2024-09-22 22:32:22
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Model Prediction   | Ground Truth   |
+=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================+====================+================+
| The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive's efforts. The packaging seemed more than adequate to protect the contents during transit.                                                                                                                                                                                                                                       | Negative           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics failed to meet my expectations. The delivery was delayed, and the customer support team was unresponsive and unhelpful when I tried to inquire about the status of my parcel.                                                                                                                                                                                                                                                                                                                                                                                              | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence resulted in a major inconvenience when my package was delivered to the wrong recipient. Despite providing accurate delivery information, the package ended up in the hands of someone else, and efforts to retrieve it were unsuccessful. When I contacted customer service for assistance, I was met with apathy and a lack of urgency. Their failure to rectify the situation in a timely manner is unacceptable, and I will be avoiding their services at all costs.                                                                                      | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Kudos to ExpressWay Logistics for their outstanding service! They've been an essential partner for my business, handling all our shipping needs with professionalism and efficiency. If you're looking for a reliable logistic service provider, look no further than ExpressWay Logistics.                                                                                                                                                                                                                                                                                                     | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics offers a convenient online platform for booking shipments, and their competitive rates initially drew me in. However, the frequent delays in delivery and lack of transparency regarding additional fees have soured my experience. While their customer service representatives are polite, they often seem powerless to resolve issues effectively.This lack of care and responsibilites from the concerned teams have led me unsatisfied and irritated.I cannot trust and rely on this again.Will definitely not contact them ever for my courier deliveries!           | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Planning a destination wedding is undoubtedly a daunting task, especially when it comes to coordinating transportation for guests and wedding essentials. Fortunately, ExpressWay Logistics made this aspect of our wedding planning effortless. From the beginning, they were attentive to our needs and provided valuable insights to streamline the process.                                                                                                                                                                                                                                 | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics consistently delivers on its promises, providing fast, efficient, and reliable courier services that exceeded my expectations.                                                                                                                                                                                                                                                                                                                                                                                                                                             | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I needed to ship perishable items internationally, and ExpressWay Logistics handled the process flawlessly. They ensured that the items were properly packaged and transported in temperature-controlled conditions.                                                                                                                                                                                                                                                                                                                                                                            | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I appreciated the flexibility offered by ExpressWay Logistics in terms of delivery options. Their range of delivery speeds and service levels allowed me to choose the option that best suited my needs and budget.However, I was slightly disappointed by the lack of transparency regarding few additional fees and surcharges. I selected a specific delivery option and paid minor charges at checkout, which undermined the initial affordability of the service..                                                                                                                         | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| While ExpressWay Logistics' pricing is competitive, their lack of accountability for delays and damages is disappointing. It's frustrating to pay for a service that fails to deliver on its promises consistently.                                                                                                                                                                                                                                                                                                                                                                             | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' pricing may seem attractive at first glance, but beware of some internal hidden fees that may sometimes add up. Coupled with their reliable delivery times, it's simply not worth the hassle.But still I appreciate that my parcel got delivered safely a day after the promised window.                                                                                                                                                                                                                                                                                  | Negative           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics has been my go-to courier service for years, primarily due to their extensive network coverage and competitive pricing. However, my recent experience has left me questioning their reliability. While their user-friendly website and efficient booking process remain commendable, the repeated delays in delivery and lack of proactive communication have been disappointing.Despite these setbacks, their customer service team has always been courteous and responsive, attempting to address my concerns promptly.                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence led to the loss of my parcel, and their lack of accountability only added to the frustration. Despite filing a claim and providing evidence of the missing items, I received no assistance or compensation for the lost package. Their negligence and disregard for customer satisfaction are unacceptable, and I will be exploring legal options to recoup the value of the lost items.                                                                                                                                                                     | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was deeply disappointed by the behavior of the delivery executive from Expressway Logistics. They exhibited a complete disregard for customer satisfaction, arriving late and mishandling the package upon delivery. Despite raising my concerns with Expressway Logistics, there was no acknowledgment or resolution provided.                                                                                                                                                                                                                                                               | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| As a frequent traveler, I often use ExpressWay Logistics to ship luggage and personal belongings ahead of time. Their door-to-door service and reliable tracking make traveling a breeze.                                                                                                                                                                                                                                                                                                                                                                                                       | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics caught my attention with its promise of seamless shipping solutions and a comprehensive network. However, my recent experiences have revealed some significant shortcomings. Their online platform is disgusting and their tracking system is inaccurate.I've encountered repeated issues with inaccurate weight calculations that have led to unexpected charges. Moreover, the limited options for customizing shipment preferences have made it challenging to meet my requirements.I cannot rely upon it for my future requirements.It failed to meet my requirements. | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| The packaging provided by ExpressWay Logistics is always secure and well-protected, and ensured that my items arrive in perfect condition.                                                                                                                                                                                                                                                                                                                                                                                                                                                      | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| My recent experience with ExpressWay Logistics left me thoroughly impressed. I had a highly time-sensitive shipment that required delicate handling and precise coordination. Despite the challenges, ExpressWay rose to the occasion, demonstrating an unparalleled level of professionalism and efficiency. Their team meticulously planned every aspect of the shipment, from packaging to transportation, ensuring that no detail was overlooked.                                                                                                                                           | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was initially impressed by ExpressWay Logistics' promise of reliability and affordability, but my recent encounters have been less than satisfactory. Their website is so irritating and their drivers are too harsh, the repeated delays in delivery have been frustrating. Additionally, the discovery of hidden fees within their pricing structure has left me feeling misled. In addition to these challenges, their customer service team has been inattentive and disrespectful, which has degraded the overall experience.Not at all recommend!                                       | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Thumbs up to ExpressWay for their prompt responses and reliable shipping solutions. Impressed!                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I had a terrible experience with ExpressWay Logistics. Not only was my pickup delayed, but the packaging was not done intactly, and the customer support team was unresponsive when I tried to raise the issue.                                                                                                                                                                                                                                                                                                                                                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
Micro F1 Score: 0.9048
Correct Predictions: 19/21
Accuracy Percentage: 90.48%
Total Reviews: 21
Positive Predictions: 8 (38.10%)
Negative Predictions: 13 (61.90%)
Positive Ground Truths: 10 (47.62%)
Negative Ground Truths: 11 (52.38%)
Evaluation End Time: 2024-09-22 22:33:15
Duration: 52.750116 seconds

Evaluation Start Time: 2024-09-22 22:33:15
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Model Prediction   | Ground Truth   |
+=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================+====================+================+
| The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive's efforts. The packaging seemed more than adequate to protect the contents during transit.                                                                                                                                                                                                                                       | Negative           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics failed to meet my expectations. The delivery was delayed, and the customer support team was unresponsive and unhelpful when I tried to inquire about the status of my parcel.                                                                                                                                                                                                                                                                                                                                                                                              | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence resulted in a major inconvenience when my package was delivered to the wrong recipient. Despite providing accurate delivery information, the package ended up in the hands of someone else, and efforts to retrieve it were unsuccessful. When I contacted customer service for assistance, I was met with apathy and a lack of urgency. Their failure to rectify the situation in a timely manner is unacceptable, and I will be avoiding their services at all costs.                                                                                      | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Kudos to ExpressWay Logistics for their outstanding service! They've been an essential partner for my business, handling all our shipping needs with professionalism and efficiency. If you're looking for a reliable logistic service provider, look no further than ExpressWay Logistics.                                                                                                                                                                                                                                                                                                     | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics offers a convenient online platform for booking shipments, and their competitive rates initially drew me in. However, the frequent delays in delivery and lack of transparency regarding additional fees have soured my experience. While their customer service representatives are polite, they often seem powerless to resolve issues effectively.This lack of care and responsibilites from the concerned teams have led me unsatisfied and irritated.I cannot trust and rely on this again.Will definitely not contact them ever for my courier deliveries!           | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Planning a destination wedding is undoubtedly a daunting task, especially when it comes to coordinating transportation for guests and wedding essentials. Fortunately, ExpressWay Logistics made this aspect of our wedding planning effortless. From the beginning, they were attentive to our needs and provided valuable insights to streamline the process.                                                                                                                                                                                                                                 | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics consistently delivers on its promises, providing fast, efficient, and reliable courier services that exceeded my expectations.                                                                                                                                                                                                                                                                                                                                                                                                                                             | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I needed to ship perishable items internationally, and ExpressWay Logistics handled the process flawlessly. They ensured that the items were properly packaged and transported in temperature-controlled conditions.                                                                                                                                                                                                                                                                                                                                                                            | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I appreciated the flexibility offered by ExpressWay Logistics in terms of delivery options. Their range of delivery speeds and service levels allowed me to choose the option that best suited my needs and budget.However, I was slightly disappointed by the lack of transparency regarding few additional fees and surcharges. I selected a specific delivery option and paid minor charges at checkout, which undermined the initial affordability of the service..                                                                                                                         | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| While ExpressWay Logistics' pricing is competitive, their lack of accountability for delays and damages is disappointing. It's frustrating to pay for a service that fails to deliver on its promises consistently.                                                                                                                                                                                                                                                                                                                                                                             | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' pricing may seem attractive at first glance, but beware of some internal hidden fees that may sometimes add up. Coupled with their reliable delivery times, it's simply not worth the hassle.But still I appreciate that my parcel got delivered safely a day after the promised window.                                                                                                                                                                                                                                                                                  | Negative           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics has been my go-to courier service for years, primarily due to their extensive network coverage and competitive pricing. However, my recent experience has left me questioning their reliability. While their user-friendly website and efficient booking process remain commendable, the repeated delays in delivery and lack of proactive communication have been disappointing.Despite these setbacks, their customer service team has always been courteous and responsive, attempting to address my concerns promptly.                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence led to the loss of my parcel, and their lack of accountability only added to the frustration. Despite filing a claim and providing evidence of the missing items, I received no assistance or compensation for the lost package. Their negligence and disregard for customer satisfaction are unacceptable, and I will be exploring legal options to recoup the value of the lost items.                                                                                                                                                                     | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was deeply disappointed by the behavior of the delivery executive from Expressway Logistics. They exhibited a complete disregard for customer satisfaction, arriving late and mishandling the package upon delivery. Despite raising my concerns with Expressway Logistics, there was no acknowledgment or resolution provided.                                                                                                                                                                                                                                                               | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| As a frequent traveler, I often use ExpressWay Logistics to ship luggage and personal belongings ahead of time. Their door-to-door service and reliable tracking make traveling a breeze.                                                                                                                                                                                                                                                                                                                                                                                                       | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics caught my attention with its promise of seamless shipping solutions and a comprehensive network. However, my recent experiences have revealed some significant shortcomings. Their online platform is disgusting and their tracking system is inaccurate.I've encountered repeated issues with inaccurate weight calculations that have led to unexpected charges. Moreover, the limited options for customizing shipment preferences have made it challenging to meet my requirements.I cannot rely upon it for my future requirements.It failed to meet my requirements. | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| The packaging provided by ExpressWay Logistics is always secure and well-protected, and ensured that my items arrive in perfect condition.                                                                                                                                                                                                                                                                                                                                                                                                                                                      | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| My recent experience with ExpressWay Logistics left me thoroughly impressed. I had a highly time-sensitive shipment that required delicate handling and precise coordination. Despite the challenges, ExpressWay rose to the occasion, demonstrating an unparalleled level of professionalism and efficiency. Their team meticulously planned every aspect of the shipment, from packaging to transportation, ensuring that no detail was overlooked.                                                                                                                                           | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was initially impressed by ExpressWay Logistics' promise of reliability and affordability, but my recent encounters have been less than satisfactory. Their website is so irritating and their drivers are too harsh, the repeated delays in delivery have been frustrating. Additionally, the discovery of hidden fees within their pricing structure has left me feeling misled. In addition to these challenges, their customer service team has been inattentive and disrespectful, which has degraded the overall experience.Not at all recommend!                                       | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Thumbs up to ExpressWay for their prompt responses and reliable shipping solutions. Impressed!                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I had a terrible experience with ExpressWay Logistics. Not only was my pickup delayed, but the packaging was not done intactly, and the customer support team was unresponsive when I tried to raise the issue.                                                                                                                                                                                                                                                                                                                                                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
Micro F1 Score: 0.9048
Correct Predictions: 19/21
Accuracy Percentage: 90.48%
Total Reviews: 21
Positive Predictions: 8 (38.10%)
Negative Predictions: 13 (61.90%)
Positive Ground Truths: 10 (47.62%)
Negative Ground Truths: 11 (52.38%)
Evaluation End Time: 2024-09-22 22:34:24
Duration: 68.786404 seconds

Evaluation Start Time: 2024-09-22 22:34:24
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Model Prediction   | Ground Truth   |
+=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================+====================+================+
| The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive's efforts. The packaging seemed more than adequate to protect the contents during transit.                                                                                                                                                                                                                                       | Negative           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics failed to meet my expectations. The delivery was delayed, and the customer support team was unresponsive and unhelpful when I tried to inquire about the status of my parcel.                                                                                                                                                                                                                                                                                                                                                                                              | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence resulted in a major inconvenience when my package was delivered to the wrong recipient. Despite providing accurate delivery information, the package ended up in the hands of someone else, and efforts to retrieve it were unsuccessful. When I contacted customer service for assistance, I was met with apathy and a lack of urgency. Their failure to rectify the situation in a timely manner is unacceptable, and I will be avoiding their services at all costs.                                                                                      | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Kudos to ExpressWay Logistics for their outstanding service! They've been an essential partner for my business, handling all our shipping needs with professionalism and efficiency. If you're looking for a reliable logistic service provider, look no further than ExpressWay Logistics.                                                                                                                                                                                                                                                                                                     | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics offers a convenient online platform for booking shipments, and their competitive rates initially drew me in. However, the frequent delays in delivery and lack of transparency regarding additional fees have soured my experience. While their customer service representatives are polite, they often seem powerless to resolve issues effectively.This lack of care and responsibilites from the concerned teams have led me unsatisfied and irritated.I cannot trust and rely on this again.Will definitely not contact them ever for my courier deliveries!           | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Planning a destination wedding is undoubtedly a daunting task, especially when it comes to coordinating transportation for guests and wedding essentials. Fortunately, ExpressWay Logistics made this aspect of our wedding planning effortless. From the beginning, they were attentive to our needs and provided valuable insights to streamline the process.                                                                                                                                                                                                                                 | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics consistently delivers on its promises, providing fast, efficient, and reliable courier services that exceeded my expectations.                                                                                                                                                                                                                                                                                                                                                                                                                                             | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I needed to ship perishable items internationally, and ExpressWay Logistics handled the process flawlessly. They ensured that the items were properly packaged and transported in temperature-controlled conditions.                                                                                                                                                                                                                                                                                                                                                                            | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I appreciated the flexibility offered by ExpressWay Logistics in terms of delivery options. Their range of delivery speeds and service levels allowed me to choose the option that best suited my needs and budget.However, I was slightly disappointed by the lack of transparency regarding few additional fees and surcharges. I selected a specific delivery option and paid minor charges at checkout, which undermined the initial affordability of the service..                                                                                                                         | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| While ExpressWay Logistics' pricing is competitive, their lack of accountability for delays and damages is disappointing. It's frustrating to pay for a service that fails to deliver on its promises consistently.                                                                                                                                                                                                                                                                                                                                                                             | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' pricing may seem attractive at first glance, but beware of some internal hidden fees that may sometimes add up. Coupled with their reliable delivery times, it's simply not worth the hassle.But still I appreciate that my parcel got delivered safely a day after the promised window.                                                                                                                                                                                                                                                                                  | Negative           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics has been my go-to courier service for years, primarily due to their extensive network coverage and competitive pricing. However, my recent experience has left me questioning their reliability. While their user-friendly website and efficient booking process remain commendable, the repeated delays in delivery and lack of proactive communication have been disappointing.Despite these setbacks, their customer service team has always been courteous and responsive, attempting to address my concerns promptly.                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence led to the loss of my parcel, and their lack of accountability only added to the frustration. Despite filing a claim and providing evidence of the missing items, I received no assistance or compensation for the lost package. Their negligence and disregard for customer satisfaction are unacceptable, and I will be exploring legal options to recoup the value of the lost items.                                                                                                                                                                     | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was deeply disappointed by the behavior of the delivery executive from Expressway Logistics. They exhibited a complete disregard for customer satisfaction, arriving late and mishandling the package upon delivery. Despite raising my concerns with Expressway Logistics, there was no acknowledgment or resolution provided.                                                                                                                                                                                                                                                               | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| As a frequent traveler, I often use ExpressWay Logistics to ship luggage and personal belongings ahead of time. Their door-to-door service and reliable tracking make traveling a breeze.                                                                                                                                                                                                                                                                                                                                                                                                       | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics caught my attention with its promise of seamless shipping solutions and a comprehensive network. However, my recent experiences have revealed some significant shortcomings. Their online platform is disgusting and their tracking system is inaccurate.I've encountered repeated issues with inaccurate weight calculations that have led to unexpected charges. Moreover, the limited options for customizing shipment preferences have made it challenging to meet my requirements.I cannot rely upon it for my future requirements.It failed to meet my requirements. | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| The packaging provided by ExpressWay Logistics is always secure and well-protected, and ensured that my items arrive in perfect condition.                                                                                                                                                                                                                                                                                                                                                                                                                                                      | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| My recent experience with ExpressWay Logistics left me thoroughly impressed. I had a highly time-sensitive shipment that required delicate handling and precise coordination. Despite the challenges, ExpressWay rose to the occasion, demonstrating an unparalleled level of professionalism and efficiency. Their team meticulously planned every aspect of the shipment, from packaging to transportation, ensuring that no detail was overlooked.                                                                                                                                           | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was initially impressed by ExpressWay Logistics' promise of reliability and affordability, but my recent encounters have been less than satisfactory. Their website is so irritating and their drivers are too harsh, the repeated delays in delivery have been frustrating. Additionally, the discovery of hidden fees within their pricing structure has left me feeling misled. In addition to these challenges, their customer service team has been inattentive and disrespectful, which has degraded the overall experience.Not at all recommend!                                       | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Thumbs up to ExpressWay for their prompt responses and reliable shipping solutions. Impressed!                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I had a terrible experience with ExpressWay Logistics. Not only was my pickup delayed, but the packaging was not done intactly, and the customer support team was unresponsive when I tried to raise the issue.                                                                                                                                                                                                                                                                                                                                                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
Micro F1 Score: 0.9048
Correct Predictions: 19/21
Accuracy Percentage: 90.48%
Total Reviews: 21
Positive Predictions: 8 (38.10%)
Negative Predictions: 13 (61.90%)
Positive Ground Truths: 10 (47.62%)
Negative Ground Truths: 11 (52.38%)
Evaluation End Time: 2024-09-22 22:35:18
Duration: 54.484005 seconds

Evaluation Start Time: 2024-09-22 22:35:18
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Model Prediction   | Ground Truth   |
+=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================+====================+================+
| The delivery executive assigned by ExpressWay Logistics was courteous and professional during the delivery process. They tried their best to handle the package with care.Unfortunately, the package arrived with slight damage despite the delivery executive's efforts. The packaging seemed more than adequate to protect the contents during transit.                                                                                                                                                                                                                                       | Negative           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics failed to meet my expectations. The delivery was delayed, and the customer support team was unresponsive and unhelpful when I tried to inquire about the status of my parcel.                                                                                                                                                                                                                                                                                                                                                                                              | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence resulted in a major inconvenience when my package was delivered to the wrong recipient. Despite providing accurate delivery information, the package ended up in the hands of someone else, and efforts to retrieve it were unsuccessful. When I contacted customer service for assistance, I was met with apathy and a lack of urgency. Their failure to rectify the situation in a timely manner is unacceptable, and I will be avoiding their services at all costs.                                                                                      | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Kudos to ExpressWay Logistics for their outstanding service! They've been an essential partner for my business, handling all our shipping needs with professionalism and efficiency. If you're looking for a reliable logistic service provider, look no further than ExpressWay Logistics.                                                                                                                                                                                                                                                                                                     | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics offers a convenient online platform for booking shipments, and their competitive rates initially drew me in. However, the frequent delays in delivery and lack of transparency regarding additional fees have soured my experience. While their customer service representatives are polite, they often seem powerless to resolve issues effectively.This lack of care and responsibilites from the concerned teams have led me unsatisfied and irritated.I cannot trust and rely on this again.Will definitely not contact them ever for my courier deliveries!           | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Planning a destination wedding is undoubtedly a daunting task, especially when it comes to coordinating transportation for guests and wedding essentials. Fortunately, ExpressWay Logistics made this aspect of our wedding planning effortless. From the beginning, they were attentive to our needs and provided valuable insights to streamline the process.                                                                                                                                                                                                                                 | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics consistently delivers on its promises, providing fast, efficient, and reliable courier services that exceeded my expectations.                                                                                                                                                                                                                                                                                                                                                                                                                                             | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I needed to ship perishable items internationally, and ExpressWay Logistics handled the process flawlessly. They ensured that the items were properly packaged and transported in temperature-controlled conditions.                                                                                                                                                                                                                                                                                                                                                                            | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I appreciated the flexibility offered by ExpressWay Logistics in terms of delivery options. Their range of delivery speeds and service levels allowed me to choose the option that best suited my needs and budget.However, I was slightly disappointed by the lack of transparency regarding few additional fees and surcharges. I selected a specific delivery option and paid minor charges at checkout, which undermined the initial affordability of the service..                                                                                                                         | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| While ExpressWay Logistics' pricing is competitive, their lack of accountability for delays and damages is disappointing. It's frustrating to pay for a service that fails to deliver on its promises consistently.                                                                                                                                                                                                                                                                                                                                                                             | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' pricing may seem attractive at first glance, but beware of some internal hidden fees that may sometimes add up. Coupled with their reliable delivery times, it's simply not worth the hassle.But still I appreciate that my parcel got delivered safely a day after the promised window.                                                                                                                                                                                                                                                                                  | Negative           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics has been my go-to courier service for years, primarily due to their extensive network coverage and competitive pricing. However, my recent experience has left me questioning their reliability. While their user-friendly website and efficient booking process remain commendable, the repeated delays in delivery and lack of proactive communication have been disappointing.Despite these setbacks, their customer service team has always been courteous and responsive, attempting to address my concerns promptly.                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics' incompetence led to the loss of my parcel, and their lack of accountability only added to the frustration. Despite filing a claim and providing evidence of the missing items, I received no assistance or compensation for the lost package. Their negligence and disregard for customer satisfaction are unacceptable, and I will be exploring legal options to recoup the value of the lost items.                                                                                                                                                                     | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was deeply disappointed by the behavior of the delivery executive from Expressway Logistics. They exhibited a complete disregard for customer satisfaction, arriving late and mishandling the package upon delivery. Despite raising my concerns with Expressway Logistics, there was no acknowledgment or resolution provided.                                                                                                                                                                                                                                                               | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| As a frequent traveler, I often use ExpressWay Logistics to ship luggage and personal belongings ahead of time. Their door-to-door service and reliable tracking make traveling a breeze.                                                                                                                                                                                                                                                                                                                                                                                                       | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| ExpressWay Logistics caught my attention with its promise of seamless shipping solutions and a comprehensive network. However, my recent experiences have revealed some significant shortcomings. Their online platform is disgusting and their tracking system is inaccurate.I've encountered repeated issues with inaccurate weight calculations that have led to unexpected charges. Moreover, the limited options for customizing shipment preferences have made it challenging to meet my requirements.I cannot rely upon it for my future requirements.It failed to meet my requirements. | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| The packaging provided by ExpressWay Logistics is always secure and well-protected, and ensured that my items arrive in perfect condition.                                                                                                                                                                                                                                                                                                                                                                                                                                                      | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| My recent experience with ExpressWay Logistics left me thoroughly impressed. I had a highly time-sensitive shipment that required delicate handling and precise coordination. Despite the challenges, ExpressWay rose to the occasion, demonstrating an unparalleled level of professionalism and efficiency. Their team meticulously planned every aspect of the shipment, from packaging to transportation, ensuring that no detail was overlooked.                                                                                                                                           | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I was initially impressed by ExpressWay Logistics' promise of reliability and affordability, but my recent encounters have been less than satisfactory. Their website is so irritating and their drivers are too harsh, the repeated delays in delivery have been frustrating. Additionally, the discovery of hidden fees within their pricing structure has left me feeling misled. In addition to these challenges, their customer service team has been inattentive and disrespectful, which has degraded the overall experience.Not at all recommend!                                       | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Thumbs up to ExpressWay for their prompt responses and reliable shipping solutions. Impressed!                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | Positive           | Positive       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| I had a terrible experience with ExpressWay Logistics. Not only was my pickup delayed, but the packaging was not done intactly, and the customer support team was unresponsive when I tried to raise the issue.                                                                                                                                                                                                                                                                                                                                                                                 | Negative           | Negative       |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
Micro F1 Score: 0.9048
Correct Predictions: 19/21
Accuracy Percentage: 90.48%
Total Reviews: 21
Positive Predictions: 8 (38.10%)
Negative Predictions: 13 (61.90%)
Positive Ground Truths: 10 (47.62%)
Negative Ground Truths: 11 (52.38%)
Evaluation End Time: 2024-09-22 22:36:36
Duration: 77.380863 seconds



##**Step 5: Observation and Insights and Business perspective (3 Marks)**
1. Observation and insigts : It observed that 
Zero shot macro F1 score = 0.9047619047619048, 
Few shot macro F1 Score = 0.9523809523809523. 
Based on this we recommend few shot prompt is giving better results , few shot approaches is recommended for this company.
Positive Predictions: 8 (38.10%)
Negative Predictions: 13 (61.90%)
Positive Ground Truths: 10 (47.62%)
Negative Ground Truths: 11 (52.38%)
